In [ ]:
import os
import getpass
os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter Your OpenAI API Key:")

# 🤖 Agents in LangChain

**Agents** are the driving force in LangChain, smart systems that use language models to work with other digital tools, perfect for tasks like Q&A, API interactions, and more.

### 🆚 Agents vs. Chains

- **Agents** dynamically decide on actions using language models.
- **Chains** have a predetermined sequence of actions.

Agents are about smart decision-making and interacting with the digital environment, while chains are about a pre-set flow of information.

### 🛠️ Tools and Toolkits

- **Tools** are like an agent's skills, specific to tasks like data retrieval.
- **Toolkits** bundle these skills for more complex tasks.

### 🧠 The Role of Memory

- Memory gives agents the ability to remember and use past interactions, making them smarter over time.



In [ ]:
from langchain_openai import ChatOpenAI

open_ai_model = "gpt-5-nano-2025-08-07" 
# instantiate the llm
llm = ChatOpenAI(model=open_ai_model,temperature=0.0)

# Giving the Agent Tools


 - 🛠️ 🔧 Tools are integrated into agents either all at once during setup  or individually as needed .

 - 🧰 Opt for LangChain's ready-made tools or craft your own custom tools to meet your unique needs.

You can find the available native tools [here](https://docs.langchain.com/oss/python/integrations/tools).

In [15]:
from langchain_community.tools import DuckDuckGoSearchRun, WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

# DuckDuckGo search
search = DuckDuckGoSearchRun()

# Wikipedia
wikipedia = WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(wiki_client=None))

# equip it with some tools
tools = [search, wikipedia]

# Create the agent

**Setting Up an Agent:**

The model is the reasoning engine of your agent. It can be specified in multiple ways, supporting both static and dynamic model selection.

1. **Static Model** 🛠️: Static models are configured once when creating the agent and remain unchanged throughout execution. This is the most common and straightforward approach.

2. **Dynamic model** 🚀: Dynamic models are selected at runtime based on the current state and context. This enables sophisticated routing logic and cost optimization.


In [16]:
query = """
Who is the current Chief AI Scientist at Meta AI? When was he born? Where was he born? What's the current temperature there?
"""

In [17]:
# initialize the agent
from langchain.agents import create_agent


agent = create_agent(
    model=llm,
    tools=tools,
    system_prompt="You are a helpful assistant that can search for information using available tools."
)

# Run with messages format
result = agent.invoke({
    "messages": [{"role": "user", "content": query}]
})

print(result)

{'messages': [HumanMessage(content="\nWho is the current Chief AI Scientist at Meta AI? When was he born? Where was he born? What's the current temperature there?\n", additional_kwargs={}, response_metadata={}, id='d36b6662-0ee8-47fa-baee-ca6a10e49597'), AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 675, 'prompt_tokens': 257, 'total_tokens': 932, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 640, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-5-nano-2025-08-07', 'system_fingerprint': None, 'id': 'chatcmpl-CcfOCRARMR3IAT421KRh5BN8ArSN7', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--988583ec-600f-450d-94b6-fc48fa63c913-0', tool_calls=[{'name': 'duckduckgo_search', 'args': {'query': 'Who is the Chief AI Scientist at Meta AI

### Dynamic model

[Dynamic agent model](https://docs.langchain.com/oss/python/langchain/agents#dynamic-model)

In [18]:
from langchain.agents import create_agent
from langchain.agents.middleware import AgentMiddleware
from langchain.messages import AIMessage
from langchain_openai import ChatOpenAI


basic_model = ChatOpenAI(model="gpt-5-nano-2025-08-07")
advanced_model = ChatOpenAI(model="gpt-5-mini-2025-08-07")


class DynamicModelMiddleware(AgentMiddleware):

    def wrap_model_call(self, request, handler):
        if len(request.state["messages"]) > self.messages_threshold:
            request.model = advanced_model
        else:
            request.model = basic_model

        return handler(request)

    def __init__(self, messages_threshold: int) -> None:
        self.messages_threshold = messages_threshold

agent = create_agent(
    model=basic_model,
    tools=tools,
    middleware=[DynamicModelMiddleware(messages_threshold=2)] # was 5 changed to 2 for testing
)


In [ ]:

queries = [
    "What is the capital of France?",
    "Who wrote 'To Kill a Mockingbird'?",
    "Explain the theory of relativity.",
    "What are the benefits of functional programming?",
    "How does a blockchain work?",
    "Can you summarize the plot of '1984' by George Orwell?",
    "What are the implications of quantum computing?",
]

# First few queries use basic_model
for i in range(7):
    query = queries[i]
    result = agent.invoke({
        "messages": [{"role": "user", "content": query}]
    }, config={"configurable": {"session_id": "1234"}})
    print(f"Query {i}: {result}")

# After 5+ messages, advanced_model kicks in
result = agent.invoke({
    "messages": [{"role": "user", "content": "What are the latest advancements in AI research?"}]
}, config={"configurable": {"session_id": "1234"}})

print(result)

Query 0: {'messages': [HumanMessage(content='What is the capital of France?', additional_kwargs={}, response_metadata={}, id='3bb6c9f4-0e21-42b6-80c9-07c05e5da6bf'), AIMessage(content='Paris.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 75, 'prompt_tokens': 219, 'total_tokens': 294, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 64, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-5-nano-2025-08-07', 'system_fingerprint': None, 'id': 'chatcmpl-CcfWnZglUqfpNJcug3742PB2ei4PV', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--3b637082-73cb-4159-9545-39422e55707b-0', usage_metadata={'input_tokens': 219, 'output_tokens': 75, 'total_tokens': 294, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 64}})]}
Q

In [47]:
for msg in result["messages"]:
    msg.pretty_print()

================================ Human Message =================================

what is the weather like in Wexford?
================================== Ai Message ==================================
Tool Calls:
  get_weather (call_YmnjJOtHl2e8VxKuwv5a3KeW)
 Call ID: call_YmnjJOtHl2e8VxKuwv5a3KeW
  Args:
    location: Wexford, Ireland
    units: metric
    include_forecast: False
================================= Tool Message =================================
Name: get_weather

Current weather in Wexford, Ireland: 72 degrees M
================================== Ai Message ==================================
Tool Calls:
  get_weather (call_WP0STYcZ1LnOIB2HU09R2Z3r)
 Call ID: call_WP0STYcZ1LnOIB2HU09R2Z3r
  Args:
    location: Wexford, Ireland
    units: metric
    include_forecast: False
================================= Tool Message =================================
Name: get_weather

Current weather in Wexford, Ireland: 72 degrees M
================================== Ai Message =======

**Weather tool Example:**

In [37]:
from langchain.tools import tool

weather_schema = {
    "type": "object",
    "properties": {
        "location": {"type": "string"},
        "units": {"type": "string"},
        "include_forecast": {"type": "boolean"}
    },
    "required": ["location", "units", "include_forecast"]
}

@tool(args_schema=weather_schema)
def get_weather(location: str, units: str = "celsius", include_forecast: bool = False) -> str:
    """Get current weather and optional forecast."""
    temp = 22 if units == "celsius" else 72
    result = f"Current weather in {location}: {temp} degrees {units[0].upper()}"
    if include_forecast:
        result += "\nNext 5 days: Sunny"
    return result

## Accessing Context

[Read first](https://docs.langchain.com/oss/python/langchain/tools#accessing-context)

#### Using Context:

Static runtime context represents immutable data like user metadata, tools, and database connections that are passed to an application at the start of a run via the context argument to invoke. 

*This data does not change during execution.*

[Reference](https://docs.langchain.com/oss/python/concepts/context#static-runtime-context)

In [ ]:
# type: ignore

from dataclasses import dataclass
from langchain.agents import create_agent
from langchain.agents.middleware import dynamic_prompt, ModelRequest
from langchain_openai import ChatOpenAI 


@dataclass
class ContextSchema:
    user_name: str

@dynamic_prompt
def personalized_prompt(request: ModelRequest) -> str:  
    user_name = request.runtime.context.user_name
    
    return f"You are a helpful assistant. Address the user as {user_name}."


model = ChatOpenAI(model="gpt-5-nano-2025-08-07")

agent = create_agent(
    model=model,
    tools=[get_weather],
    middleware=[personalized_prompt],
    context_schema=ContextSchema
)

result = agent.invoke(
    {"messages": [{"role": "user", "content": "what is the weather like in Wexford?"}]},
    context=ContextSchema(user_name="Mo Che")  
)

In [46]:
for msg in result["messages"]:
    msg.pretty_print()

================================ Human Message =================================

what is the weather like in Wexford?
================================== Ai Message ==================================
Tool Calls:
  get_weather (call_YmnjJOtHl2e8VxKuwv5a3KeW)
 Call ID: call_YmnjJOtHl2e8VxKuwv5a3KeW
  Args:
    location: Wexford, Ireland
    units: metric
    include_forecast: False
================================= Tool Message =================================
Name: get_weather

Current weather in Wexford, Ireland: 72 degrees M
================================== Ai Message ==================================
Tool Calls:
  get_weather (call_WP0STYcZ1LnOIB2HU09R2Z3r)
 Call ID: call_WP0STYcZ1LnOIB2HU09R2Z3r
  Args:
    location: Wexford, Ireland
    units: metric
    include_forecast: False
================================= Tool Message =================================
Name: get_weather

Current weather in Wexford, Ireland: 72 degrees M
================================== Ai Message =======

#### Using State:

Dynamic runtime context represents mutable data that can evolve during a single run and is managed through the LangGraph state object. This includes conversation history, intermediate results, and values derived from tools or LLM outputs. In LangGraph, the state object acts as short-term memory during a run.

[Reference](https://docs.langchain.com/oss/python/concepts/context#dynamic-runtime-context)

In [ ]:
from langchain.agents import create_agent
from langchain.agents.middleware import dynamic_prompt, ModelRequest
from langchain.agents import AgentState
from langchain_openai import ChatOpenAI 


class CustomState(AgentState):  
    user_name: str

@dynamic_prompt
def personalized_prompt(request: ModelRequest) -> str:  
    user_name = request.state.get("user_name", "User")
    return f"You are a helpful assistant. User's name is {user_name}"

model = ChatOpenAI(model="gpt-5-nano-2025-08-07")

agent = create_agent(
    model=model,
    state_schema=CustomState,  
    middleware=[personalized_prompt],  
)

result = agent.invoke({
    "messages": [{"role": "user", "content": "hi!"}],
    "user_name": "Mo Che"
}) # type: ignore

print(result)

> Turning on memory: Memory is a powerful feature that allows you to persist the agent’s state across multiple invocations. Otherwise, the state is scoped only to a single run. See next notebook.

### [Dynamic system prompt](https://docs.langchain.com/oss/python/langchain/agents#dynamic-system-prompt)

*See also [Tool Strategy][Tool Strategy]*

[Tool Strategy]: https://docs.langchain.com/oss/python/langchain/agents#toolstrategy

### [Life-cycle Context](https://docs.langchain.com/oss/python/langchain/context-engineering#life-cycle-context)

# Exercise

Create an agent that uses a different LLM model based on whether the user has a paid subscription or not.


In [ ]:

from dataclasses import dataclass
from langchain.agents import create_agent
from langchain.agents.middleware import dynamic_prompt, ModelRequest
from langgraph.runtime import Runtime

@dataclass
class Context:  
    subscription: str = "free"  # or "paid"

# use gpt-5-mini-2025-08-07 for paid users, gpt-5-nano-2025-08-07 for free users




# 🔄 **Agents and Memory**

### **Stateless Agent Characteristics**

- **Current Limitation**: Inability to recall past interactions by default.

### **Enabling Memory**:

- **Method**: Integrate previous chat history into the agent.

- **Key Requirement**: The chat history variable must be named "chat_history" for compatibility with the current prompt.

- **Flexibility**: Altering the prompt
allows for different variable naming.

🔑 **Takeaway**: Adjusting the prompt or incorporating chat history enables stateful interactions, enhancing the agent's capability to remember and utilize past interactions.

In [39]:
# type: ignore

from langchain.agents import create_agent
from langchain_openai import ChatOpenAI 
from langgraph.checkpoint.memory import InMemorySaver  


model = ChatOpenAI(model="gpt-5-nano-2025-08-07")

agent = create_agent(
    model=model,
    tools=[get_weather],
    checkpointer=InMemorySaver()
)

result = agent.invoke(
    {"messages": [{"role": "user", "content": "what is the weather like in Wexford?"}]},
    config={"configurable": {"thread_id": "1"}}
)

result

{'messages': [HumanMessage(content='what is the weather like in Wexford?', additional_kwargs={}, response_metadata={}, id='0bebb529-4ac9-4c28-af50-f0b718f8f954'),
  AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 485, 'prompt_tokens': 145, 'total_tokens': 630, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 448, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-5-nano-2025-08-07', 'system_fingerprint': None, 'id': 'chatcmpl-CchEmQRBdoLIAAZ6egR1QJH64fAr2', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--b5afa9ec-c9cb-4374-9232-fb0aac956bf6-0', tool_calls=[{'name': 'get_weather', 'args': {'location': 'Wexford, Ireland', 'units': 'metric', 'include_forecast': False}, 'id': 'call_YmnjJOtHl2e8VxKuwv5a3KeW', 'type': 'tool_call'}], usage

In [41]:
result = agent.invoke(
    {"messages": [{"role": "user", "content": "in celsius please"}]},
    config={"configurable": {"thread_id": "1"}} # try without thread_id
)
result

{'messages': [HumanMessage(content='what is the weather like in Wexford?', additional_kwargs={}, response_metadata={}, id='0bebb529-4ac9-4c28-af50-f0b718f8f954'),
  AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 485, 'prompt_tokens': 145, 'total_tokens': 630, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 448, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-5-nano-2025-08-07', 'system_fingerprint': None, 'id': 'chatcmpl-CchEmQRBdoLIAAZ6egR1QJH64fAr2', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--b5afa9ec-c9cb-4374-9232-fb0aac956bf6-0', tool_calls=[{'name': 'get_weather', 'args': {'location': 'Wexford, Ireland', 'units': 'metric', 'include_forecast': False}, 'id': 'call_YmnjJOtHl2e8VxKuwv5a3KeW', 'type': 'tool_call'}], usage

#### Config

The `thread_id` in config is used by:

`InMemorySaver` to retrieve the correct chat history for that session
Middleware functions to track conversation context (like your `dynamic_model_selection` function)

**Without config**:
Each invoke call is stateless and independent, and memory/history features won't work.

**With config**:
Multiple invoke calls with the same session_id can share state and maintain conversation history across calls.